# DSCI-598 Capstone
## Maryville University
### November - December 2023
### Alison Hawke

## Principal Component Analysis using a Random Forest model

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

np.random.seed(1)

# Exploring the data set

The Forest Cover data set has zero null values, and each cover type is represented exactly equally in the test set. This implies the data was manufactured to create a good training set.

In [ ]:
train = pd.read_csv('/kaggle/input/forest-cover-type-prediction/train.csv', dtype = str)
train.shape

In [ ]:
train.head

In [ ]:
test = pd.read_csv('/kaggle/input/forest-cover-type-prediction/test.csv', dtype = str)
test.shape

In [ ]:
train.isnull().sum()

In [ ]:
train.value_counts(['Cover_Type']).sort_index()

In [ ]:
X_num_train = train.loc[:, ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
                   'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                   'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']].values
X_cat_train = train.loc[:, ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 
                      'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 
                      'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 
                      'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 
                      'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 
                      'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
                      'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29',
                      'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
                      'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
                      'Soil_Type40',]].values
y = train.loc[:, 'Cover_Type'].values

print('Numerical Feature Array Shape:   ', X_num_train.shape)
print('Categorical Feature Array Shape: ', X_cat_train.shape)
print('Label Array Shape:               ', y.shape)

In [ ]:
# join arrays
X = np.hstack((X_num_train, X_cat_train))

print('Feature Array Shape:', X.shape)

In [ ]:
X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify = y)
X_valid, X_test, y_valid, y_test = train_test_split(X_hold, y_hold, test_size = 0.2, random_state = 1, stratify = y_hold)

# Principal Component Analysis (PCA)

Principal Component Analysis reduces the number of dimensions in the data and allows you to eliminate features that do not contribute to the predictions.

In [ ]:
X = StandardScaler().fit_transform(X)
X.shape

In [ ]:
print("Scaled features mean: ", round(np.mean(X), 4))
print("Scaled features standard deviation: ", round(np.std(X), 4))

In [ ]:
#pca_cover = PCA(n_components = 4)
pca_cover = PCA(n_components = 2)
principalComponents_cover = pca_cover.fit_transform(X)

In [ ]:
#principal_cover_df = pd.DataFrame(data = principalComponents_cover
#             , columns = ['principal component 1', 'principal component 2', 
#                          'principal component 3', 'principal component 4'])

principal_cover_df = pd.DataFrame(data = principalComponents_cover
             , columns = ['principal component 1', 'principal component 2'])
principal_cover_df.head

In [ ]:
print('Explained variation per principal component: {}'.format(pca_cover.explained_variance_ratio_))

# Random Forest model

Using parameters from my previous investigation for the best random forest model, n_estimators = 1000, max_depth = 25.

In [ ]:
%%time

best_rf_train_acc = []
best_rf_valid_acc = []

np.random.seed(1)
best_forest = RandomForestClassifier(n_estimators = 1000, max_depth = 25)
best_forest.fit(X_train, y_train)

best_rf_train_acc.append(best_forest.score(X_train, y_train))
best_rf_valid_acc.append(best_forest.score(X_valid, y_valid))

## Displaying the Gini importance of each of the features

In [ ]:
feature_labels = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
                   'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                   'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
                   'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 
                      'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 
                      'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 
                      'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 
                      'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 
                      'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
                      'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29',
                      'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
                      'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
                      'Soil_Type40',]

for feature in zip(feature_labels, best_forest.feature_importances_):
    print(feature)

In [ ]:
# Create a selector object that will use the random forest classifier 
# to identify features that have an importance above a threshold

# sfm = SelectFromModel(best_forest, threshold = 0.0001) = 0.73597 v12
# sfm = SelectFromModel(best_forest, threshold = 0.001) = 0.73807 v5
# sfm = SelectFromModel(best_forest, threshold = 0.01) = 0.71823 v9
# sfm = SelectFromModel(best_forest, threshold = 0.1) = 0.41248 v11
sfm = SelectFromModel(best_forest, threshold = 0.001)

# Train the selector
sfm.fit(X_train, y_train)

After some experimentation, the best threshold for Gini importance of features is 0.001. This investigation involved running and submitting multiple versions of the same model to Kaggle with different thresholds in order to find the best accuracy on the test set.

In [ ]:
X_num_test = test.loc[:, ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
                   'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
                   'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']].values
X_cat_test = test.loc[:, ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 
                      'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 
                      'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 
                      'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 
                      'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 
                      'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
                      'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29',
                      'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
                      'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39',
                      'Soil_Type40',]].values

X_test = np.hstack((X_num_test, X_cat_test))
X_test.shape

Transform the data to create a new dataset containing only the most important features. This transformation must be applied to both the training X and test X data.

In [ ]:
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)
X_important_valid = sfm.transform(X_valid)

### Using 1000 estimators gave the most accurate model with a depth of 25. Keeping the number of estimators the same, the next variation is the model depth

In [ ]:
%%time

# Create a new random forest classifier for the most important features
# Vary the number of estimators from 100 to 2000
# 1000 estimators produced the most accurate model with a depth of 25

rf_train_acc = []
rf_valid_acc = []
depth_range = range(2, 30)

for d in depth_range:
    np.random.seed(1)
    temp_forest = RandomForestClassifier(n_estimators = 1000, max_depth = d)
    temp_forest.fit(X_important_train, y_train)
    rf_train_acc.append(temp_forest.score(X_important_train, y_train))
    rf_valid_acc.append(temp_forest.score(X_important_valid, y_valid))

rf_idx = np.argmax(rf_valid_acc)
rf_opt_depth = depth_range[rf_idx]

print('Random Forest model:')
print('Optimal value for max_depth:           ', rf_opt_depth)
print('Training Accuracy for Optimal Model:   ', round(rf_train_acc[rf_idx], 4))
print('Validation Accuracy for Optimal Model: ', round(rf_valid_acc[rf_idx], 4))

In [ ]:
plt.figure(figsize = [8, 6])

plt.plot(depth_range, rf_train_acc, label = 'Training')
plt.plot(depth_range, rf_valid_acc, label = 'Validation')

plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Training vs Validation data')
plt.legend()
plt.show()

In [ ]:
pca_rf_train_acc = []
pca_rf_valid_acc = []

# Using the optimal estimator and depth parameters
pca_cover = RandomForestClassifier(n_estimators = 1000, max_depth = 26)

# Train the new classifier on the new dataset containing the most important features
pca_cover.fit(X_important_train, y_train)

pca_rf_train_acc.append(pca_cover.score(X_important_train, y_train))
pca_rf_valid_acc.append(pca_cover.score(X_important_valid, y_valid))

In [ ]:
print('PCA random forest model:')
print('Training Accuracy:   ', round(pca_rf_train_acc[0], 4))
print('Validation Accuracy: ', round(pca_rf_valid_acc[0], 4))

# Submission 

In [ ]:
X_important_test.shape

In [ ]:
%%time

# Apply the classifier to the test data
y_important_pred = pca_cover.predict(X_important_test)

In [ ]:
Id = np.asarray(pd.read_csv('/kaggle/input/forest-cover-type-prediction/test.csv')['Id'])#

#print(f'Id: ', Id.shape)
#print('y_pred: ', y_important_pred.shape)

submission = pd.DataFrame({'Id':Id, 'Cover_Type':y_important_pred})
submission.to_csv('submission.csv', header = True, index = False)